In [1]:
import re
import tabula
import requests
import json
import csv
import boto3
import pandas as pd
import numpy as np
from data_extraction import DataExtractor as de
from database_utils import DatabaseConnector as dc
from datetime import datetime as dt

extr = de()
conn = dc()

In [95]:
dh = extr.read_rds_table(conn, conn.list_db_tables()[2])            #Orders table
dh = dh.drop(['level_0', 'first_name', 'last_name', '1'], axis=1)
#dh.drop_duplicates(['card_number'], inplace=True)
print(len(dh))

120123


In [178]:
df = extr.retrieve_product_data()
display(df.head(20))

,Unnamed: 0,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6kg,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,1,Tiffany's World Day Out At The Park,£12.99,0.48kg,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,2,Tiffany's World Pups Picnic Playset,£7.00,590g,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,3,Tiffany's World Wildlife Park Adventures,£12.99,540g,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,4,Cosatto Cosy Dolls Pram,£30.00,1.91kg,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a
5,5,Cocomelon Magnetic Scribbler,£12.99,0.91kg,toys-and-games,9022505507373,2007-02-07,cdd568e3-9657-412b-b3a6-52f6adff966b,Still_avaliable,g3-7974400s
6,6,Peppa Pig House Felties,£7.00,0.46kg,toys-and-games,5997977455405,2009-01-24,58112753-a53f-4696-ab8a-7186c7e4a57e,Still_avaliable,U3-5148457q
7,7,Paw Patrol Walkie Talkies,£12.99,0.38kg,toys-and-games,58555395171,2013-08-13,797d133a-0989-4707-914d-7c043bcbbd86,Still_avaliable,q8-5387483Z
8,8,Peppa Pig 3D Walkie Talkies,£12.99,0.38kg,toys-and-games,9155372675699,2005-05-26,a6c23718-0e25-4419-8018-600c44e5e3df,Still_avaliable,i4-2651057I
9,9,Disney Encanto Magical Casa Madrigal,£89.99,8.981kg,toys-and-games,2740880244498,2003-08-05,2e78e9a4-2b77-4b59-95e5-d60ee01ef183,Still_avaliable,b6-6234052g


In [175]:
df = extr.retrieve_product_data()
df['weight_check'] = df['weight'].str.contains('x')
dg = df['weight'][df['weight_check'] == True].str.replace('[xg]', '', regex=True).str.split(expand=True)
dg['weight'] = dg[0].astype(int) * dg[1].astype(int)
dg['weight'] = dg['weight'].astype(str) + 'g'
df.update(dg)
df = df.drop(['weight_check'], axis=1)
grams = df[df.loc[:, 'weight'].str.contains('([^k][g])|[ml]') == True]
grams.loc[:, 'weight'] = grams.loc[:, 'weight'].str.removesuffix('g')
grams.loc[:, 'weight'] = grams.loc[:, 'weight'].str.removesuffix('g .')
grams.loc[:, 'weight'] = grams.loc[:, 'weight'].str.removesuffix('ml')
grams.loc[:, 'weight'] = pd.to_numeric(grams.loc[:, 'weight']) / 1000
df.update(grams)
ounces = df[df.loc[:, 'weight'].str.contains('oz') == True]
ounces.loc[:, 'weight'] = ounces.loc[:, 'weight'].str.removesuffix('oz')
ounces.loc[:, 'weight'] = round(ounces.loc[:, 'weight'].astype(float) / 35.274, 2)
df.update(ounces)
kg = df[df.loc[:, 'weight'].str.contains('kg') == True]
kg.loc[:, 'weight'] = kg.loc[:, 'weight'].str.removesuffix('kg')
df.update(kg)
df.rename(columns={'weight': 'weight_kg'}, inplace=True)
df = df.drop('Unnamed: 0', axis=1)
df = df[df['removed'].isin(['Still_avaliable', 'Removed']) == True]
df.loc[:, 'product_price'] = df.loc[:, 'product_price'].str.replace('£', '')
df.rename(columns={'product_price': 'product_price_£'}, inplace=True)
df['date_added'] = pd.to_datetime(df['date_added']).dt.date
df['product_price_£'] = df['product_price_£'].astype(float)
df['weight_kg'] = df['weight_kg'].astype(float)
df.reset_index(drop=True, inplace=True)
df['products_key'] = [number for number in range(len(df))]

C:\Users\srayn\AppData\Local\Temp\ipykernel_12116\1543574544.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  grams = df[df.loc[:, 'weight'].str.contains('([^k][g])|[ml]') == True]
C:\Users\srayn\AppData\Local\Temp\ipykernel_12116\1543574544.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grams.loc[:, 'weight'] = grams.loc[:, 'weight'].str.removesuffix('g')
C:\Users\srayn\AppData\Local\Temp\ipykernel_12116\1543574544.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [177]:
display(df.head(20))

,product_name,product_price_£,weight_kg,category,EAN,date_added,uuid,removed,product_code,products_key
0,FurReal Dazzlin' Dimples My Playful Dolphin,39.99,1.6000,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h,0
1,Tiffany's World Day Out At The Park,12.99,0.4800,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l,1
2,Tiffany's World Pups Picnic Playset,7.00,0.5900,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v,2
3,Tiffany's World Wildlife Park Adventures,12.99,0.5400,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n,3
4,Cosatto Cosy Dolls Pram,30.00,1.9100,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a,4
5,Cocomelon Magnetic Scribbler,12.99,0.9100,toys-and-games,9022505507373,2007-02-07,cdd568e3-9657-412b-b3a6-52f6adff966b,Still_avaliable,g3-7974400s,5
6,Peppa Pig House Felties,7.00,0.4600,toys-and-games,5997977455405,2009-01-24,58112753-a53f-4696-ab8a-7186c7e4a57e,Still_avaliable,U3-5148457q,6
7,Paw Patrol Walkie Talkies,12.99,0.3800,toys-and-games,58555395171,2013-08-13,797d133a-0989-4707-914d-7c043bcbbd86,Still_avaliable,q8-5387483Z,7
8,Peppa Pig 3D Walkie Talkies,12.99,0.3800,toys-and-games,9155372675699,2005-05-26,a6c23718-0e25-4419-8018-600c44e5e3df,Still_avaliable,i4-2651057I,8
9,Disney Encanto Magical Casa Madrigal,89.99,8.9810,toys-and-games,2740880244498,2003-08-05,2e78e9a4-2b77-4b59-95e5-d60ee01ef183,Still_avaliable,b6-6234052g,9


In [172]:
card_list = list(df['card_number'])
print(len(card_list))

15284


In [173]:
orders_list = list(dh['card_number'].unique())
print(len(orders_list))

15284


In [174]:
difference_result = []

for number in orders_list:
    if number not in card_list:
        difference_result.append(number)

print(len(difference_result))

0


In [154]:
print(difference_result)

[4145662949755970, 4150158657691, 3583218047699840, 6011887212979720, 6011592595426290, 213164174481249, 6521667118670950, 2274876960738170, 6575242255783560, 4617848619884830000, 4546958607493620, 4671576682730, 4009798496502850, 30043592114296, 3534672147045390, 4466728925689730000, 4540308816293220000, 564128564718, 344884797663251, 3509093146400700, 6011369698621020, 4264610789837620, 180015756135190, 3503542374654200, 4158723133970670, 4338477003933360, 564984589809, 4350714812589620, 6537258455501990, 6520005527716070, 4092454363906180, 3555499065337720, 213133430820417, 30382459793101, 60111111913046, 675964295934, 6580954508196020, 379544388859480, 639051308133, 676197624627, 30436408634905, 30366098644395, 4507220961742, 30240385709155, 4511109856138500000, 3562364872068970, 3550482967595170, 30092025361172, 371842803993779, 3520167590602520, 375231530935095, 30022003678867, 38586217093206, 4971858637664481, 30535888793394, 4064806653749, 6011982654011120, 2226666455714860, 18

In [169]:
print(card_list)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '4145662949755970', '4150158657691', '3583218047699840', '6011887212979720', '6011592595426290', '213164174481249', '6521667118670950', '2274876960738170', '6575242255783560', '4617848619884830000', '4546958607493620', '4671576682730', '4009798496502850', '30043592114296', '3534672147045390', '4466728925689730000', '4540308816293220000', '564128564718', '344884797663251', '3509093146400700', '6011369698621020', '4264610789837620', '1800157561351

In [157]:
display(df[df['card_number'] == 3583218047699840])

,card_number,expiry_date,card_provider,date_payment_confirmed,cards_key
